In [1]:
# путь до данных на компьютере
path = 'train_data/'

In [2]:
import os
import pandas as pd
import tqdm


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразовывает их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой нужно начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                              if filename.startswith('train')])
    print(dataset_paths)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)

    return pd.concat(res).reset_index(drop=True)

In [3]:
def prepare_transactions_dataset(path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                 save_to_path=None, verbose: bool=False):
    """
    возвращает готовый pd.DataFrame с признаками, на которых можно учить модель для целевой задачи
    path_to_dataset: str
        путь до датасета с партициями
    num_parts_to_preprocess_at_once: int
        количество партиций, которые будут одновременно держаться и обрабатываться в памяти
    num_parts_total: int
        общее количество партиций, которые нужно обработать
    save_to_path: str
        путь до папки, в которой будет сохранён каждый обработанный блок в .parquet-формате; если None, то не будет сохранён
    verbose: bool
        логирует каждую обрабатываемую часть данных
    """
    preprocessed_frames = []

    for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once,
                                                             verbose=verbose)


   #здесь должен быть препроцессинг данных
    
    
   #записываем подготовленные данные в файл
        if save_to_path:
            block_as_str = str(step)
            if len(block_as_str) == 1:
                block_as_str = '00' + block_as_str
            else:
                block_as_str = '0' + block_as_str
            transactions_frame.to_parquet(os.path.join(save_to_path, f'processed_chunk_{block_as_str}.parquet'))

        preprocessed_frames.append(transactions_frame)
    return pd.concat(preprocessed_frames)

In [4]:
data = prepare_transactions_dataset(path, num_parts_to_preprocess_at_once=12, num_parts_total=12) 

C:\Users\Yana Eliseeva\AppData\Local\Temp\ipykernel_5612\1399563192.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),


Transforming transactions data:   0%|          | 0/1 [00:00<?, ?it/s]

['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq', 'train_data/train_target.csv']


C:\Users\Yana Eliseeva\AppData\Local\Temp\ipykernel_5612\3059785415.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):


Reading dataset with pandas:   0%|          | 0/12 [00:00<?, ?it/s]

chunk_path train_data/train_data_0.pq
chunk_path train_data/train_data_1.pq
chunk_path train_data/train_data_10.pq
chunk_path train_data/train_data_11.pq
chunk_path train_data/train_data_2.pq
chunk_path train_data/train_data_3.pq
chunk_path train_data/train_data_4.pq
chunk_path train_data/train_data_5.pq
chunk_path train_data/train_data_6.pq
chunk_path train_data/train_data_7.pq
chunk_path train_data/train_data_8.pq
chunk_path train_data/train_data_9.pq


In [5]:
data.to_csv('dataset/dataset.csv', index = False)

In [1]:
import pandas as pd
df = pd.read_csv('dataset/dataset.csv')

In [2]:
# Преобразуем все колонки, кроме id , в int8
df_id = df.id
df=df.drop('id', axis = 1)
df=df.astype('int8')
df['id'] = df_id
df.head()

,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,...,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,id
0,1,18,9,2,3,16,10,11,3,3,...,3,3,4,1,3,4,1,0,0,0
1,2,18,9,14,14,12,12,0,3,3,...,0,0,4,1,3,4,1,0,0,0
2,3,18,9,4,8,1,11,11,0,5,...,0,0,4,1,2,3,1,1,1,0
3,4,4,1,9,12,16,7,12,2,3,...,3,3,4,1,3,1,1,0,0,0
4,5,5,12,15,2,11,12,10,2,3,...,3,3,4,1,3,4,1,0,0,0


In [3]:
df.to_csv('dataset/dataset_int8.csv', index = False)

In [4]:
#сделаем  небольшую выборку для разведочного анализа
import pandas as pd
df_target = pd.read_csv('train_data/train_target.csv')
df_target.shape

(3000000, 2)

In [6]:
df_target_sample = df_target.sample(n=30000, random_state = 1)

# Посмотрим на соотношение значений таргета
print(f'Исходный df_target: {df_target.flag.value_counts()}')
print(f'df_target_sample: {df_target_sample.flag.value_counts()}')

Исходный df_target: flag
0    2893558
1     106442
Name: count, dtype: int64
df_target_sample: flag
0    28948
1     1052
Name: count, dtype: int64


In [8]:
df_target_sample = df_target_sample.set_index('id')
df_target_sample.head()

,flag
id,
2896109,0
2381153,0
1028690,0
1945977,0
2812693,0


In [10]:
# соберем датасет со всеми строками,относящимися к id, попавшими в семпл-выборку 
#(пока не аггрегируем -хочу иметь возможность попробовать на семпле разную аггрегацию)
df_joined = df.join(df_target_sample, on = 'id', how = 'inner')
df_joined.head()

,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,...,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,id,flag
783,1,11,9,16,9,15,6,3,1,3,...,3,4,1,3,4,1,0,0,100,0
784,2,2,9,14,7,10,8,11,5,5,...,3,4,1,2,3,1,0,0,100,0
785,3,7,9,16,9,1,11,11,0,2,...,3,4,1,2,4,1,0,0,100,0
786,4,19,10,14,7,10,4,16,1,2,...,3,4,1,2,4,1,0,0,100,0
787,5,12,0,4,8,1,11,4,2,3,...,3,4,1,2,3,1,1,1,100,0


In [12]:
df_joined.shape

(262177, 62)

In [24]:
df_joined.id.nunique()

30000

In [26]:
df_joined.to_csv('dataset/dataset_sample.csv', index = False)